# Intro to the ChatGPT API

By the end of this talk, you will be able to:
- interact with ChatGPT in a jupyter-notebook/google colab/VSCode
- summarize text
- perform sentiment analysis
- develop a chatbot

## Intro
- GPT is short for Generative Pre-trained Transformer model
    - it provides text outputs in response to text inputs (prompts)
    - prompts have four goals:
        - ask a question 
        - provide detailed instructions
        - provide some examples of how to successfully complete a task
        - provide domain knowledge ChatGPT needs to know to complete a task
- ChatGPT is a large language model (LLM) improved by reinforcement learning with human feedback (RLHF)
- You can interact with it in two ways:
    - web interface: https://chat.openai.com/
        - free or \$20/month for a ChatGPT Plus plan 
    - API access mostly for developers to build chat-based applications
        - token-based, I paid less than \$0.05 to develop and test code for this talk


## Warning #1
- ChatGPT is a third party software
- Everything you ask and the responses you receive are collected and stored by OpenAI
- DO NOT share sensitive data and personally identifiable info (PII) with AI tools such as ChatGPT, Bard, Github Copilot, etc.
- No Level 2 and 3
<center><img src="datariskclassification.png" width="600"></center>


## Warning #2
- ChatGPT is not reproducible!
- It does have a parameter to set the degree of randomness of the output called `temperature`
- But setting it to 0 still does not guarantee reproducability!
- ChatGPT is continually updated based on user feedback 

<font color='LIGHTGRAY'>By the end of this talk, you will be able to:</font>
- **interact with ChatGPT in a jupyter-notebook/google colab/VSCode**
- <font color='LIGHTGRAY'>summarize text</font>
- <font color='LIGHTGRAY'>perform sentiment analysis</font>
- <font color='LIGHTGRAY'>develop a chatbot</font>

## The get_completion() function

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompts, roles = ['user'], model = "gpt-3.5-turbo", temperature = 0, n = 1, verbose = False):
    '''
    prompts: str or list
        If str, it is a single prompt. If a list, it contains a list of strings in a message history.
    roles: list, default is ['user']
        A list of roles in a message history, usually the elements are 'user' or 'assistant'.
    model: str, default is "gpt-3.5-turbo"
        The specific model version to be used for generating the response.
    temperature: float between 0 and 2, default is 0
        The degree of randomness of the model's output.
    n: int, default is 1
        The number of completions to generate.
    verbose: boolean, default is False
        If True, the input messages and the full response in JSON format are printed. 

    Returns: str, list, or JSON object
        The model's response. It is a string if n = 1 and verbose == False. 
        It is a list if n > 1 and verbose == False. It is a JSON object if verbose == True.
    
    Use the prompts and roles lists to provide message history. 
    This is useful if chatGPT needs context for a successful response.

    Example:
    
    prompts = ['Tell me a joke.', 'Why did the chicken cross the road?', 'I don’t know, why did the chicken cross the road?']
    roles = ['user','assistant','user'] 

    The response will be the punchline of the joke.
    '''

    # check inputs and prepare messages
    if type(prompts) == str:
        messages = [{'role':'user','content':prompts}]
    elif type(prompts) == list:
        if len(roles) != len(prompts):
            raise ValueError('Lengths of roles and prompts are not equal!')
        # combine roles and prompts
        messages = [{"role":roles[i],"content":prompts[i]} for i in range(len(roles))] 
    else:
        raise ValueError('prompts is neither a string nor a list!')
        
    if verbose:
        print(messages)

    # query ChatGPT
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        n = n,
        temperature=temperature, 
    )

    if verbose:
        # return the full response as a JSON object
        return response
    else:
        if n == 1: 
            # return the only response as a string
            return response.choices[0].message["content"]
        else:
            # return all responses as a list of strings
            return [choice.message["content"] for choice in response.choices]

In [3]:
# example of a simple prompt, no message history
prompt = 'Tell me a joke!'
response = get_completion(prompt)
print(response)

Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!


In [4]:
roles = ['user','assistant','user']
prompts = ['Tell me a joke.', 'Why did the chicken cross the road?', 'I don’t know, why did the chicken cross the road?']

response = get_completion(prompts,roles)
print(response)

To get to the other side!


<font color='LIGHTGRAY'>By the end of this talk, you will be able to:</font>
- <font color='LIGHTGRAY'>interact with ChatGPT in a jupyter-notebook/google colab/VSCode</font>
- **summarize text**
- <font color='LIGHTGRAY'>perform sentiment analysis</font>
- <font color='LIGHTGRAY'>develop a chatbot</font>

## Summarize text

In [5]:
import pandas as pd
import numpy as np

# read in small dataset of drug reviews
df = pd.read_csv('data/drugsComTrain_raw.tsv',sep='\t')

# grab the index of the longest review
indx = np.argmax(df['review'].str.len())

df['review'].iloc[indx]


'"Abilify 20 mg.\r\nI am a patient diagnosed with disorganized schizophrenia, depression,  schizoaffective disorder, bipolar.  I have experienced a sensitivity to my emotions, as well as how I react to my feelings.  I really don&#039;t feel out of the normal with any &quot;sexual frustration&quot;...and I wouldn&#039;t say this has increased/decreased.  I feel less anxious on the medication, and seemingly more at ease with myself when I take this medication.  It is a hard step to have people telling you that you actually do better on this pill, when half your life ago, you didn&#039;t have &quot;mental illness&quot;, and suddenly you become someone else.   My weight did fluxuate when on this drug.  But I feel it was due to stress factors outside of a regular environment.  I feel if you place yourself in good surroundings and support you do much better at being the person you are here to be.  I am so much happier in life with a routine.  I feel like I only referred to enlongated periods

In [6]:
# text to summarize
prod_review = df['review'].iloc[indx]

In [7]:
prompt = f"""
Your task is to generate a short summary of a drug \
review from a pharma site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The reviewer, a patient with multiple mental health diagnoses, shares their experience with Abilify. They mention feeling less anxious and more at ease on the medication, but also express concerns about its long-term effects and potential risks during pregnancy.


In [8]:
# let's shift focus to certain aspects
prompt = f"""
Your task is to generate a short summary of a drug \
review from a pharma site. 

Summarize the review below focusing on aspects 
related to dosage, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


The reviewer has been taking Abilify 20 mg for their mental illnesses and has experienced less anxiety and improved well-being. They have concerns about the long-term effects and potential side effects, but overall feel that the medication has been beneficial for them.


In [9]:
# let's shift focus to certain aspects
prompt = f"""
Your task is to generate a short summary of a drug \
review from a pharma site. 

Summarize the review below focusing on the side effects
the patient experienced, 
delimited by triple backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


The patient experienced sensitivity to emotions and weight fluctuation while taking Abilify. They also expressed concern about the long-term effects and potential risks during pregnancy.


<font color='LIGHTGRAY'>By the end of this talk, you will be able to:</font>
- <font color='LIGHTGRAY'>interact with ChatGPT in a jupyter-notebook/google colab/VSCode</font>
- <font color='LIGHTGRAY'>summarize text</font>
- **perform sentiment analysis**
- <font color='LIGHTGRAY'>develop a chatbot</font>

## Sentiment analysis

In [10]:
prod_review2 = df['review'].iloc[4]

print(prod_review2)

"Suboxone has completely turned my life around.  I feel healthier, I&#039;m excelling at my job and I always have money in my pocket and my savings account.  I had none of those before Suboxone and spent years abusing oxycontin.  My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction.  All that is history.  If you&#039;re ready to stop, there&#039;s a good chance that suboxone will put you on the path of great life again.  I have found the side-effects to be minimal compared to oxycontin.  I&#039;m actually sleeping better.   Slight constipation is about it for me.  It truly is amazing. The cost pales in comparison to what I spent on oxycontin."


In [11]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{prod_review2}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [12]:
# single word response
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive", \
"neutral", or "negative".

Review text: '''{prod_review2}'''
"""
response = get_completion(prompt)
print(response)

positive


In [13]:
# single number response
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single number between -1 and +1,
where -1 is negative, 0 is neutral, and +1 is positive.

Review text: '''{prod_review2}'''
"""
response = get_completion(prompt)
print(response)

0.9


In [14]:
# identify emotions
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \

Give your answer as either yes or no.

Review text: '''{prod_review2}'''
"""
response = get_completion(prompt)
print(response)

No


<font color='LIGHTGRAY'>By the end of this talk, you will be able to:</font>
- <font color='LIGHTGRAY'>interact with ChatGPT in a jupyter-notebook/google colab/VSCode</font>
- <font color='LIGHTGRAY'>summarize text</font>
- <font color='LIGHTGRAY'>perform sentiment analysis</font>
- **develop a chatbot**

## Chatbots

Let's create a chatbot for a pizza restaurant!

First, ChatGPT needs context!

We describe what it should do and what domain knowledge it needs.

This is done by a `system` role.

In [15]:
roles = ['system']
prompts = ["""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
If it's a pickup, tell the customer that the address is 123 Main Street.
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
Once you took the order, repond with a simple "Goodbye!".
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""] 

In [16]:
# we will use a while loop because we can't know in advance how many interactions we will need
order_pending = True
nr_interactions  = 0

while order_pending:

    # we want to make sure the loop stops eventually
    if nr_interactions == 100:
        raise ValueError('The order was not created successfully!')

    # the chatbot's message
    chatbot_msg = get_completion(prompts,roles)

    # the user's response
    prompts.append(chatbot_msg)
    roles.append('assistant')

    if 'Goodbye!' in chatbot_msg:
        # the final message is printed and we exit the loop
        print(chatbot_msg)
        order_pending = False
    else:

        # the customer's response
        customer_msg = input(chatbot_msg)

        # update prompts and roles
        prompts.append(customer_msg)
        roles.append('user')
    
        nr_interactions += 1

Hello! Welcome to our pizza restaurant. How can I assist you today? I'd like to order one pizza please.
Sure! What type of pizza would you like? We have pepperoni, cheese, and eggplant. Cheese.
Great choice! What size would you like for your cheese pizza? We have small, medium, and large. medium.
Perfect! So you'd like a medium cheese pizza. Is there anything else you'd like to add to your order? no thanks
Alright, just to confirm, you'd like a medium cheese pizza. Is that correct? yes
Got it! Is this order for pickup or delivery? pickup
Great! Your order will be ready for pickup. You can pick it up at our restaurant located at 123 Main Street. Is there anything else I can assist you with? No thanks.
Alright, your order for a medium cheese pizza is confirmed for pickup at 123 Main Street. Your total comes to $10.95. How would you like to pay? credit card
Perfect! Please provide me with your credit card information, including the card number, expiration date, and CVV code. 23492934, 345

Thank you for providing your credit card information. Your payment has been processed successfully. Your order will be ready for pickup shortly. If you have any further questions or need any assistance, feel free to ask. Goodbye!


In [17]:
# let's create a json summary that can be shared with other systems

roles.append('system')
prompts.append('create a json summary of the previous food order. \
Itemize the price for each item. \
The fields should be 1) pizza, include size 2) list of toppings \
3) list of drinks, include size  4) list of sides include size  \
5)total price ')

response = get_completion(prompts,roles)
print(response)

{
  "pizza": {
    "type": "cheese",
    "size": "medium",
    "price": 10.95
  },
  "toppings": [],
  "drinks": [],
  "sides": [],
  "total_price": 10.95
}


## Thanks for your attention!

I hope by now you know how to:
- interact with ChatGPT in a jupyter-notebook/google colab/VSCode,
- summarize text,
- perform sentiment analysis,
- develop a chatbot.

Check out the prompt engineering course on [deeplearning.ai](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) to learn more!